1. Install the following libraries:
- msal
- httpx

2. Create Azure app:
- Go to portal.azure.com
- Nav to app registration
- Registre your app
- Running on web -> https://localhost.com:1992
- Manage -> Certificates & Secrets -> New Client Secret

3. Add Client Secret, App ID to your environment

In [7]:
import os
import msal
import httpx
import webbrowser
from dotenv import load_dotenv

In [12]:
def get_access_token(application_id, client_secret, scopes):
    client = msal.ConfidentialClientApplication(
        client_id = application_id,
        client_credential = client_secret,
        authority = "https://login.microsoftonline.com/consumers/"
    )

    refresh_token = None
    if os.path.exists("refresh_token.txt"):
        with open("refresh_token.txt", "r") as f:
            refresh_token = f.read().strip()

    if refresh_token:
        token_response = client.acquire_token_by_refresh_token(refresh_token, scopes)
    else:
        auth_request_url = client.get_authorization_request_url(scopes)
        webbrowser.open(auth_request_url)
        auth_code = input("Enter the auth code: ")

        if not auth_code:
            raise ValueError("No auth code provided")
        
        token_response = client.acquire_token_by_authorization_code(
            code = auth_code, 
            scopes = scopes
        )

    if "access_token" in token_response:
        if "refresh_token" in token_response:
            with open("refresh_token.txt", "w") as f:
                f.write(token_response["refresh_token"])
        return token_response["access_token"]
    else:
        raise Exception("No access token in response")

In [15]:
load_dotenv()
application_id = os.getenv("azure_app_id")
client_secret = os.getenv("azure_client_secret")
scopes = ["User.Read", "Mail.ReadWrite", "Mail.Send"]

MS_GRAPH_BASE_URL = "https://graph.microsoft.com/v1.0"

access_token = get_access_token(application_id, client_secret, scopes)
headers = {
    "Authorization": "Bearer " + access_token
}

In [21]:
def draft_message(recipient, subject):
    message = {
        "subject": subject,
        "body": {
            "contentType": "Text",
            "content": "This is a test email using the Microsoft Graph API"
        },
        "toRecipients": [
            {
                "emailAddress": {
                    "address": recipient
                }
            }
        ],
        "ccRecipients": [
            {
                "emailAddress": {
                    "address": "carlos.toruno@gmail.com"
                }
            }
        ]
    }
    return message

In [22]:
endpoint = f"{MS_GRAPH_BASE_URL}/me/sendMail"

In [ ]:
message = {
    "message": draft_message("dgranada@equilibriumbdc.com", "TESTING API")
}
response = httpx.post(endpoint, headers=headers, json=message)
if response.status_code != 202:
    raise Exception(f"Failed to send email: {response.text}")
else:
    print("Email sent successfully")

Email sent successfully
